<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/chuongmep/aps-toolkit/blob/dev/APSToolkitPython/Tutorials/06.%20Data%20Visualization%20-%20Analyst%20BIM%20Model.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Run in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/chuongmep/aps-toolkit/blob/dev/APSToolkitPython/Tutorials/06.%20Data%20Visualization%20-%20Analyst%20BIM%20Model.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

<table align="left">
  <tr>
    <td>Author(s)</td>
    <td><a href="https://github.com/chuongmep" id="author-link">Chuong Ho</a></td>
    <!-- <td><a href="<your repo>" id="github-link">YourName</a></td> -->
  </tr>
</table>

In [1]:
%pip install aps_toolkit --upgrade

In [2]:
from aps_toolkit import Auth
from aps_toolkit import PropDbReaderRevit
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

#### Auth Via Google Colab

In [ ]:
# from google.colab import userdata
# client_id = userdata.get('APS_CLIENT_ID')
# client_secret = userdata.get('APS_CLIENT_SECRET')
# auth = Auth(client_id, client_secret)
# token = auth.auth2leg()

In [4]:
urn = "dXJuOmFkc2sud2lwcHJvZDpmcy5maWxlOnZmLk9kOHR4RGJLU1NlbFRvVmcxb2MxVkE_dmVyc2lvbj0yOA"
token = Auth().auth2leg()
prop_reader = PropDbReaderRevit(urn, token)

## Categories Analysis

In [5]:
cates = prop_reader.get_all_categories()

In [6]:
# get all categories string 
dict_categories = prop_reader.get_all_categories()
categories = list(dict_categories.values())
# visualize with x,y : x : category, y : count element
x = []
y = []
for cate in categories:
    df_cate = prop_reader.get_data_by_category(cate)
    if(df_cate.shape[0] == 0):
        continue
    y.append(df_cate.shape[0])
    x.append(cate)
x_sorted, y_sorted = zip(*sorted(zip(x, y), key=lambda pair: pair[1], reverse=True))
plt.figure(figsize=(20,10))
sns.barplot(x=list(x_sorted), y=list(y_sorted))
plt.xticks(rotation=90)
plt.xlabel('Category')
plt.ylabel('Count')
plt.title('Count of Elements in Each Category')
plt.show()

In [7]:
y_top10 = list(x_sorted)[:10]
x_top10 = list(y_sorted)[:10]
plt.figure(figsize=(10, 5))
sns.barplot(x=x_top10, y=y_top10, palette='viridis')
plt.xlabel('Count', fontsize=12)
plt.ylabel('Category', fontsize=12)
plt.title('Top 10 Categories with Most Elements', fontsize=14)
plt.xticks(rotation=45) 
plt.tight_layout()  
plt.show()

Why have so many categories don't relate to data ? 
Try check materials 

## Materials

In [8]:
pd.set_option('display.max_columns', 8)
pd.set_option('display.max_rows', 20)
df_materials = prop_reader.get_data_by_categories(["Materials"])
df_materials

In [9]:
df_materials.info()

In [10]:
df_materials.describe()

In [11]:
df_materials.shape

In [ ]:
# see all name of materials
materials = df_materials["Name"].unique()
materials.sort()
materials

In [ ]:
material_parameters = df_materials.columns
material_parameters

In [ ]:
# group by material name and get the count of each material
df_materials_grouped = df_materials.groupby("Name").size().reset_index(name='Count')
df_materials_grouped = df_materials_grouped.sort_values(by='Count', ascending=False)
df_materials_grouped

In [ ]:
# see who create the most materials by see values column Edited by
people = df_materials["Edited by"].unique()
people

## Furniture

In [ ]:
df_furniture = prop_reader.get_data_by_category("Furniture")
df_furniture

In [ ]:
df_furniture.info()

In [ ]:
df_furniture.describe()

In [ ]:
# which parameters for furniture
df_furniture.columns

In [ ]:
## see all the parameters contains 'IFC' and value of them 
df_furniture[df_furniture.columns[df_furniture.columns.str.contains('IFC')]]
# => look may be need input ifc value for furniture

In [ ]:
# try with walls but it same 
df_walls = prop_reader.get_data_by_category("Walls")
df_walls[df_walls.columns[df_walls.columns.str.contains('IFC')]]

## Rooms

In [ ]:
df_rooms = prop_reader.get_data_by_category("Rooms")
df_rooms

In [ ]:
df_rooms.columns

In [ ]:
## see room name,area,level,volume,...
df_rooms[["ElementId","Name","Number", "Area", "Level", "Volume","Comments"]]

In [ ]:
# see any room Area > 100 
df_rooms[df_rooms["Area"] > 100]
# => no room area > 100

In [ ]:
# see ifc value of room is it filled of empty
df_rooms[df_rooms.columns[df_rooms.columns.str.contains('IFC')]]